In [28]:
import re, os, time
import pandas as pd
import datetime
import numpy as np
from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score
from matplotlib import pyplot as plt
# import plotData # helper function in starter code package


from sktime.transformations.panel.padder import PaddingTransformer
from sktime.classification.compose import ClassifierPipeline, ComposableTimeSeriesForestClassifier
from sktime.transformations.panel.summarize import RandomIntervalFeatureExtractor
from sklearn.tree import DecisionTreeClassifier
# only classifier in sktime that can process unequal length data
# https://github.com/sktime/sktime/issues/3649#issuecomment-1292459843
# from sktime.alignment.dtw_python import AlignerDTW   ## NOTE THAT THIS SOMEHOW AFFECT ALL PRINT OUTPUT. NOTHING WILL BE SHOWN FOR PRINT STATEMENT AFTER YOU RUN THIS
from sktime.classification.feature_based import RandomIntervalClassifier
from sktime.classification.distance_based import KNeighborsTimeSeriesClassifier
from sktime.classification.dictionary_based import IndividualBOSS, ContractableBOSS
from sktime.classification.kernel_based import RocketClassifier
from sktime.classification.hybrid import HIVECOTEV1, HIVECOTEV2
from sktime.classification.shapelet_based import ShapeletTransformClassifier
from sktime.classification.sklearn import RotationForest

from sktime.dists_kernels.compose_from_align import DistFromAligner
from sktime.utils.slope_and_trend import _slope
from sklearn.pipeline import Pipeline
# https://www.sktime.org/en/stable/api_reference/auto_generated/sktime.transformations.panel.catch22.Catch22.html
from sktime.transformations.panel.catch22 import Catch22

from sktime.classification.interval_based import CanonicalIntervalForest,DrCIF,RandomIntervalSpectralEnsemble,SupervisedTimeSeriesForest,TimeSeriesForestClassifier

# identify classifiers that support unequal length
from sktime.registry import all_estimators




In [29]:
data_root = "..\\cleanedData\\"

## Combining Data

In [59]:
# combine into single df
df_combined_subject = {'subject':[],'run':[], 'normalised_resp':[], 'difficulty':[]}
for root, dirs, files in os.walk(data_root):
  for file in files:
      if "lslshimmerresp" in file:
        difficulty = re.search("mV_(\d\d\w)", file).group(1)
        subject = re.search("_(cp\d+)_", file).group(1)
        run = re.search("_(\d).csv", file).group(1)
        df_temp = pd.read_csv(os.path.join(root, file))
        # We perform scaler on EACH subject as they are independent of each other 
        scaler = StandardScaler()
        resp_series = pd.Series(scaler.fit_transform(df_temp[['respiration_trace_mV']]).flatten())

        df_combined_subject['subject'].append(subject)
        df_combined_subject['run'].append(run)
        df_combined_subject['normalised_resp'].append(resp_series)
        df_combined_subject['difficulty'].append(difficulty)
df_combined_subject = pd.DataFrame(df_combined_subject)

In [60]:
# save to pickle rather than csv to preserve the nested series inside the dataframe
df_combined_subject.to_pickle(data_root+"df_combined_subject.pkl", protocol=4)

# Modelling

In [3]:
df_combined_subject = pd.read_pickle(data_root+"df_combined_subject.pkl")

In [13]:
# Identify rows that is 0 length
# empty_row = []
# for i in range(len(df_combined)):
#   temp = df_combined.iloc[i,0]
#   if len(temp[temp==0.0]) or len(temp[temp==0]):
#     empty_row.append(i)

# df_combined.drop(empty_row, inplace=True)

In [16]:
%%time
# perform train test split according by subject
# split into 5 different folds for CV
from sklearn.model_selection import GroupKFold
X_train, X_test, y_train, y_test = [], [], [], []
gss = GroupKFold(n_splits=5)
for train, test in gss.split(df_combined_subject["normalised_resp"], df_combined_subject["difficulty"], df_combined_subject["subject"]):
  X_train.append(df_combined_subject.loc[train,["normalised_resp"]])
  X_test.append(df_combined_subject.loc[test,["normalised_resp"]])
  y_train.append(df_combined_subject.loc[train,"difficulty"].astype("string"))
  y_test.append(df_combined_subject.loc[test,"difficulty"].astype("string"))

CPU times: total: 0 ns
Wall time: 13 ms


In [15]:
# X_train, X_test, y_train, y_test = train_test_split(df_combined_subject["normalised_resp"], df_combined_subject["difficulty"], random_state=42)

In [16]:
# X_train = pd.DataFrame(X_train)
# X_test = pd.DataFrame(X_test)
# y_train = y_train.astype("string")
# y_test = y_test.astype("string")

In [33]:
model_result = {
  "classifier":[],
  "accuracy_score":[],
  "AUC_score":[],
  "F1_score":[],
  "runtime(s)":[],
}

def get_class(class_list, prob_list):
  idx = list(prob_list).index(max(prob_list))
  return class_list[idx]

def log_result(classifier_name, class_list, y_test, y_pred_proba, runtime):
  y_pred = []
  for y_list in y_pred_proba:
    y_pred.append(get_class(class_list, y_list))
  acc = accuracy_score(y_test, y_pred)
  auc = roc_auc_score(y_test, y_pred_proba, multi_class='ovr')
  f1 = f1_score(y_test, y_pred, average='micro')
  model_result["classifier"].append(classifier_name)
  model_result["accuracy_score"].append(acc)
  model_result["AUC_score"].append(auc)
  model_result["F1_score"].append(f1)
  model_result["runtime(s)"].append(runtime)

  display(pd.DataFrame(model_result))
  pd.DataFrame(model_result).to_csv(data_root+"respiration_split_pilot_result.csv")

In [34]:
def run_model(classifier_name,classifier, padding = True,  fold = 0):
  start = time.time()
  
  # set up pipeline
  if padding:
    clf = PaddingTransformer() * classifier()
  else:
    clf = classifier()
  
  clf.fit(X_train[fold], y_train[fold])
  y_pred_proba = clf.predict_proba(X_test[fold])
  end = time.time()

  log_result(classifier_name, clf.classes_, y_test[fold], y_pred_proba, end-start)

## Classification using catch22

Refer to respiration_split_pilot_pycaret.ipynb

## RandomIntervalClassifier
extract at random interval and perform Rotation forest with 200 trees

In [23]:
run_model(
  "RandomIntervalClassifier", 
  lambda: RandomIntervalClassifier(n_intervals=5, n_jobs=1, random_state = 42),
  padding = True,
  fold = 0
  )

KeyboardInterrupt: 

In [28]:
run_model(
  "RandomIntervalClassifier", 
  lambda: RandomIntervalClassifier(n_intervals=5, n_jobs=1, random_state = 42),
  padding = True,
  fold = 0
  )

,classifier,accuracy_score,AUC_score,F1_score,runtime(s)
0,RandomIntervalClassifier,0.44898,0.712098,0.44898,2665.528686


## Decision Trees with mean, std, slope


In [29]:
steps = [
    ("padding",PaddingTransformer()),
    (
        "extract",
        RandomIntervalFeatureExtractor(
            n_intervals="sqrt", features=[np.mean, np.std, _slope]
        ),
    ),
    ("clf", DecisionTreeClassifier()),
]
time_series_tree = Pipeline(steps)

In [30]:
start = time.time()
time_series_tree.fit(X_train[0], y_train[0])
y_pred_proba = time_series_tree.predict_proba(X_test[0])
end = time.time()
log_result('RandomeIntervalDecisionTree',time_series_tree.classes_, y_test[0], y_pred_proba, end-start)

,classifier,accuracy_score,AUC_score,F1_score,runtime(s)
0,RandomIntervalClassifier,0.448980,0.712098,0.448980,2665.528686
1,RandomeIntervalDecisionTree,0.459184,0.669589,0.459184,4584.890491


## Individual Boss


In [31]:
run_model(
  "IndividualBOSS", 
  lambda: IndividualBOSS(),
  padding = True,
  fold = 0
  )

,classifier,accuracy_score,AUC_score,F1_score,runtime(s)
0,RandomIntervalClassifier,0.448980,0.712098,0.448980,2665.528686
1,RandomeIntervalDecisionTree,0.459184,0.669589,0.459184,4584.890491
2,IndividualBOSS,0.224490,0.516234,0.224490,2147.196138


## ContractableBoss

In [32]:
run_model(
  "ContractableBOSS", 
  lambda: ContractableBOSS(n_parameter_samples=10, max_ensemble_size=3),
  padding = True,
  fold = 0
  )

,classifier,accuracy_score,AUC_score,F1_score,runtime(s)
0,RandomIntervalClassifier,0.448980,0.712098,0.448980,2665.528686
1,RandomeIntervalDecisionTree,0.459184,0.669589,0.459184,4584.890491
2,IndividualBOSS,0.224490,0.516234,0.224490,2147.196138
3,ContractableBOSS,0.214286,0.536054,0.214286,2236.196237


## Random Interval Spectral Ensemble

In [33]:
run_model(
  "RandomIntervalSpectralEnsemble", 
  lambda: RandomIntervalSpectralEnsemble(n_estimators=50, random_state=42),
  padding = True,
  fold = 0
  )

,classifier,accuracy_score,AUC_score,F1_score,runtime(s)
0,RandomIntervalClassifier,0.448980,0.712098,0.448980,2665.528686
1,RandomeIntervalDecisionTree,0.459184,0.669589,0.459184,4584.890491
2,IndividualBOSS,0.224490,0.516234,0.224490,2147.196138
3,ContractableBOSS,0.214286,0.536054,0.214286,2236.196237
4,RandomIntervalSpectralEnsemble,0.408163,0.703726,0.408163,2305.620849


## Supervised Time Series Forest (STSF)


In [34]:
run_model(
  "SupervisedTimeSeriesForest", 
  lambda: SupervisedTimeSeriesForest(n_estimators=50, random_state=42),
  padding = True,
  fold = 0
  )

,classifier,accuracy_score,AUC_score,F1_score,runtime(s)
0,RandomIntervalClassifier,0.448980,0.712098,0.448980,2665.528686
1,RandomeIntervalDecisionTree,0.459184,0.669589,0.459184,4584.890491
2,IndividualBOSS,0.224490,0.516234,0.224490,2147.196138
3,ContractableBOSS,0.214286,0.536054,0.214286,2236.196237
4,RandomIntervalSpectralEnsemble,0.408163,0.703726,0.408163,2305.620849
5,SupervisedTimeSeriesForest,0.540816,0.776268,0.540816,2511.660846


## Canonical Interval Forest (CIF)

In [35]:
run_model(
  "CanonicalIntervalForest", 
  lambda: CanonicalIntervalForest(n_estimators=5, att_subsample_size=10, random_state=42),
  padding = True,
  fold = 0
  )

,classifier,accuracy_score,AUC_score,F1_score,runtime(s)
0,RandomIntervalClassifier,0.448980,0.712098,0.448980,2665.528686
1,RandomeIntervalDecisionTree,0.459184,0.669589,0.459184,4584.890491
2,IndividualBOSS,0.224490,0.516234,0.224490,2147.196138
3,ContractableBOSS,0.214286,0.536054,0.214286,2236.196237
4,RandomIntervalSpectralEnsemble,0.408163,0.703726,0.408163,2305.620849
5,SupervisedTimeSeriesForest,0.540816,0.776268,0.540816,2511.660846
6,CanonicalIntervalForest,0.448980,0.733673,0.448980,19594.243454


## Diverse Representation Canonical Interval Forest (DrCIF)

In [36]:
run_model(
  "DiverseRepresentationCanonicalIntervalForest", 
  lambda: DrCIF(n_estimators=5, att_subsample_size=10, random_state=42),
  padding = True,
  fold = 0
  )

,classifier,accuracy_score,AUC_score,F1_score,runtime(s)
0,RandomIntervalClassifier,0.448980,0.712098,0.448980,2665.528686
1,RandomeIntervalDecisionTree,0.459184,0.669589,0.459184,4584.890491
2,IndividualBOSS,0.224490,0.516234,0.224490,2147.196138
3,ContractableBOSS,0.214286,0.536054,0.214286,2236.196237
4,RandomIntervalSpectralEnsemble,0.408163,0.703726,0.408163,2305.620849
5,SupervisedTimeSeriesForest,0.540816,0.776268,0.540816,2511.660846
6,CanonicalIntervalForest,0.448980,0.733673,0.448980,19594.243454
7,DiverseRepresentationCanonicalIntervalForest,0.397959,0.704252,0.397959,10634.717233


## ShapeletTransformClassifier

In [37]:
run_model(
  "ShapeletTransformClassifier", 
  lambda: ShapeletTransformClassifier(
    estimator=RotationForest(n_estimators=3),
    n_shapelet_samples=100,
    max_shapelets=10,
    batch_size=20,
    ),
  padding = True,
  fold = 0
  )

,classifier,accuracy_score,AUC_score,F1_score,runtime(s)
0,RandomIntervalClassifier,0.448980,0.712098,0.448980,2665.528686
1,RandomeIntervalDecisionTree,0.459184,0.669589,0.459184,4584.890491
2,IndividualBOSS,0.224490,0.516234,0.224490,2147.196138
3,ContractableBOSS,0.214286,0.536054,0.214286,2236.196237
4,RandomIntervalSpectralEnsemble,0.408163,0.703726,0.408163,2305.620849
5,SupervisedTimeSeriesForest,0.540816,0.776268,0.540816,2511.660846
6,CanonicalIntervalForest,0.448980,0.733673,0.448980,19594.243454
7,DiverseRepresentationCanonicalIntervalForest,0.397959,0.704252,0.397959,10634.717233
8,ShapeletTransformClassifier,0.224490,0.509848,0.224490,13577.179583


## RocketClassifier

In [38]:
run_model(
  "RocketClassifier", 
  lambda: RocketClassifier(num_kernels=500),
  padding = True,
  fold = 0
  )

,classifier,accuracy_score,AUC_score,F1_score,runtime(s)
0,RandomIntervalClassifier,0.448980,0.712098,0.448980,2665.528686
1,RandomeIntervalDecisionTree,0.459184,0.669589,0.459184,4584.890491
2,IndividualBOSS,0.224490,0.516234,0.224490,2147.196138
3,ContractableBOSS,0.214286,0.536054,0.214286,2236.196237
4,RandomIntervalSpectralEnsemble,0.408163,0.703726,0.408163,2305.620849
5,SupervisedTimeSeriesForest,0.540816,0.776268,0.540816,2511.660846
6,CanonicalIntervalForest,0.448980,0.733673,0.448980,19594.243454
7,DiverseRepresentationCanonicalIntervalForest,0.397959,0.704252,0.397959,10634.717233
8,ShapeletTransformClassifier,0.224490,0.509848,0.224490,13577.179583
9,RocketClassifier,0.367347,0.623377,0.367347,3134.879266


## FAILED MODELS

### KNeighborsTimeSeriesClassifier

In [21]:
# # search for all classifiers which can handle unequal length data. This may give some
# # UserWarnings if soft dependencies are not installed.
# all_estimators(
#     filter_tags={"capability:unequal_length": True}, estimator_types="classifier"
# )

c:\Users\kyunomi\anaconda3\envs\cogpilot\lib\site-packages\sktime\utils\validation\_dependencies.py:143: UserWarning: No module named 'hcrystalball'. 'hcrystalball' is a soft dependency and not included in the base sktime installation. Please run: `pip install hcrystalball` to install the hcrystalball package. To install all soft dependencies, run: `pip install sktime[all_extras]`
  warnings.warn(msg)


[('DummyClassifier', sktime.classification.dummy._dummy.DummyClassifier),
 ('KNeighborsTimeSeriesClassifier',
  sktime.classification.distance_based._time_series_neighbors.KNeighborsTimeSeriesClassifier),
 ('TimeSeriesSVC', sktime.classification.kernel_based._svc.TimeSeriesSVC)]

In [ ]:
# aligner = AlignerDTW()
# dtw_dist = DistFromAligner(aligner)
# knclassifier = KNeighborsTimeSeriesClassifier(n_neighbors=3, distance = dtw_dist, n_jobs= -1)
# knclassifier.fit(X_train, y_train)
# y_pred = knclassifier.predict(X_test)

# log_result('KNeighborsTimeSeriesClassifier',y_test, y_pred)

# NO MEMORY

### KNeighborsTimeSeriesClassifier with padding

In [2]:
# padded_KN_pipeline = ClassifierPipeline(
#     KNeighborsTimeSeriesClassifier(n_neighbors=5, distance ="dtw", n_jobs= 1, leaf_size = 2000), 
#     [PaddingTransformer()]
# )
# padded_KN_pipeline.fit(X_train, y_train)
# y_pred = padded_KN_pipeline.predict(X_test)

# log_result('KNeighborsTimeSeriesClassifier',y_test, y_pred)

# NO MEMORY

### ComposableTimeSeriesForestClassifier
https://www.sktime.org/en/v0.8.1/examples/02_classification_univariate.html

In [ ]:
# # https://www.sktime.org/en/v0.8.1/examples/02_classification_univariate.html
# tsf_tst = PaddingTransformer() * ComposableTimeSeriesForestClassifier(
#     estimator=time_series_tree,
#     n_estimators=100,
#     # criterion="entropy",
#     bootstrap=True,
#     oob_score=True,
#     random_state=1,
#     n_jobs=-1,
# )
# tsf_tst.fit(X_train, y_train.astype("string"))

# if tsf_tst.oob_score:
#     print(tsf.oob_score_)

# y_pred_proba = tsf_tst.predict_proba(X_test)

# log_result('tsf_time_series_forest',tsf_tst.classes_, y_test, y_pred_proba)

# STOPPED PREMATURELY BECAUSE NO OUTPUT EVEN AFTER 12 hours

### HIVECOTEV1

In [39]:
# run_model(
#   "HIVECOTEV1", 
#   lambda: HIVECOTEV1(),
#   padding = True,
#   fold = 0
#   )

# STOPPED PREMATURELY BECAUSE NO OUTPUT EVEN AFTER 24 hours

# Optimising SupervisedTimeSeriesForest

In [11]:
for i in np.arange(10,300,20):
    run_model(
        "SupervisedTimeSeriesForest_"+str(i)+"_trees", 
        lambda: SupervisedTimeSeriesForest(n_estimators=i, random_state=42),
        padding = True,
        fold = 0
    )

Refer to result in \CogPilot_Challenge\cleanedData\respiration_split_pilot_stsf.csv

n_estimator of 40 gives the best AUC score

## Prepare data for merging with PPG

In [30]:
### WE DO THIS IN ORDER TO MERGE WITH PPG
# perform train test split according by subject
# split into 5 different folds for CV
from sklearn.model_selection import GroupKFold
X_train, X_test, y_train, y_test, test_subj = [], [], [], [], []
gss = GroupKFold(n_splits=5)
for train, test in gss.split(df_combined_subject["normalised_resp"][df_combined_subject["difficulty"]!="000"], df_combined_subject["difficulty"][df_combined_subject["difficulty"]!="000"], df_combined_subject["subject"][df_combined_subject["difficulty"]!="000"]):
  X_train.append(df_combined_subject.loc[train,["normalised_resp"]])
  X_test.append(df_combined_subject.loc[test,["normalised_resp"]])
  y_train.append(df_combined_subject.loc[train,"difficulty"].astype("string"))
  y_test.append(df_combined_subject.loc[test,"difficulty"].astype("string"))
  test_subj.append(df_combined_subject.loc[test,"subject"].astype("string"))

In [31]:
test_subj[0].unique()

<StringArray>
['cp004', 'cp005', 'cp014', 'cp015', 'cp023', 'cp024', 'cp039', 'cp042',
 'cp008', 'cp016', 'cp018', 'cp029', 'cp043', 'cp009', 'cp017', 'cp019',
 'cp025', 'cp030']
Length: 18, dtype: string

In [35]:
#extract y_pred out and classes
start = time.time()
clf = PaddingTransformer() * SupervisedTimeSeriesForest(n_estimators=40, random_state=42)
clf.fit(X_train[0], y_train[0])
y_pred_proba = clf.predict_proba(X_test[0])
end = time.time()

log_result(SupervisedTimeSeriesForest, clf.classes_, y_test[0], y_pred_proba, end-start)

In [ ]:
# perform prediction on all data
df_combined_all = pd.DataFrame(clf.predict_proba(df_combined_subject[['normalised_resp']]))
df_combined_all.columns = 'resp_'+clf.classes_
df_combined_all['difficulty'] = df_combined_subject['difficulty']
df_combined_all['subject'] = df_combined_subject['subject']
df_combined_all['run'] = df_combined_subject['run']

In [ ]:
df_combined_all = df_combined_all[['subject', 'run', 'resp_000', 'resp_01B', 'resp_02B', 'resp_03B', 'resp_04B',
       'difficulty']]

In [ ]:
df_combined_all.to_csv("df_combined_all.csv", index=False)

# WE CAN"T COMBINE NOW BECAUSE 1) THERE"S MISSING 000 DATA iN PPG. 2) THE GROUPKFOLD IS NOT THE SAME. THEREFORE ABLE TO DO PROPER COMPARISON

# Try looking
- filter out different subject with different resp hz then do training for them.
- do gridsearch using genetic to improve result